# Simple preprocessing pipeline

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from scipy import stats

from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict, learning_curve,\
train_test_split, GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, OneHotEncoder, FunctionTransformer, LabelEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.inspection import permutation_importance
from sklearn.metrics import plot_confusion_matrix, classification_report, precision_recall_curve
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDRegressor, SGDClassifier, Ridge, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.compose import make_column_selector

## pipeline stuff

from sklearn.pipeline import Pipeline, make_pipeline, make_union
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn import set_config; set_config(display='diagram')

In [3]:
import warnings 
warnings.filterwarnings('ignore')

In [4]:
# def_df = pd.read_csv("/home/slawa/code/code-rep0/projects/data/defaulter_data_13364.csv", index_col=[0])
# pay_df = pd.read_csv("/home/slawa/code/code-rep0/projects/data/payer_data_41940.csv", index_col=[0])

def_df = pd.read_csv("/Users/sjoerddewit/Desktop/Programming/6 Le Wagon Data Science/final_project/defaulter_data_6k_ids.csv", index_col=[0])
pay_df = pd.read_csv("/Users/sjoerddewit/Desktop/Programming/6 Le Wagon Data Science/final_project/payer_data_20k_ids.csv", index_col=[0])

def_df['default'] = 1
pay_df['default'] = 0

df = pd.concat([def_df, pay_df])

df.reset_index(inplace=True)


In [5]:
y = df['default'].reset_index(drop=True)

X = df.drop(columns=['default']).reset_index(drop=True)

Discarding columns if too highly correlated with other or too many Nans

In [6]:
cat_vars = ['B_30', 
            'B_38', 
            'D_114', 
            'D_116', 
            'D_117', 
            'D_120', 
            'D_126', 
            'D_63', 
            'D_64', 
            'D_66', 
            'D_68']

drop columns if they correlate > 95% with others

In [7]:
X_corr = X.corr()

In [8]:
X_corr = X_corr.unstack().reset_index() # Unstack correlation matrix 
X_corr.columns = ['feature_1','feature_2', 'correlation_all'] # rename columns
X_corr.sort_values(by="correlation_all",ascending=False, inplace=True) # sort by correlation
X_corr = X_corr[X_corr['feature_1'] != X_corr['feature_2']] # Remove self correlation
X_corr = X_corr.drop_duplicates(subset='correlation_all')

In [9]:
red_features = list(X_corr[abs(X_corr['correlation_all'])>=.95]['feature_1']) ## abs so we also consider the negative corrs

In [10]:
X_red = X.drop(columns=red_features) ## dropping the highly correlated columns

In [11]:
## checking whether the high correlations are gone
X_red_corr = X_red.corr()
X_red_corr = X_red_corr.unstack().reset_index() # Unstack correlation matrix 
X_red_corr.columns = ['feature_1','feature_2', 'correlation_all'] # rename columns
X_red_corr.sort_values(by="correlation_all",ascending=False, inplace=True) # sort by correlation
X_red_corr = X_red_corr[X_red_corr['feature_1'] != X_red_corr['feature_2']] # Remove self correlation
X_red_corr = X_red_corr.drop_duplicates(subset='correlation_all')

In [12]:
X_red_corr

,feature_1,feature_2,correlation_all
7572,B_14,B_15,0.925751
7244,D_58,D_74,0.921838
6860,B_13,B_12,0.919627
19428,B_33,B_2,0.911653
909,S_3,S_7,0.904065
...,...,...,...
8802,B_16,B_18,-0.784312
8861,B_16,B_33,-0.784688
580,B_2,B_20,-0.786150
11586,S_15,S_8,-0.789232


In [13]:
len(red_features) ## we removed 13 columns

13

drop columns with nans if in __both__ groups > 80% nans

In [14]:
nan_threshold= 0.8 ## adjust the hardcoded values

In [15]:
def_nans = def_df.isna().sum()/len(def_df) 

In [16]:
def_nans_80 = def_nans[def_nans >= nan_threshold].index

In [17]:
pay_nans = pay_df.isna().sum()/len(pay_df)

In [18]:
pay_nans_80 = pay_nans[pay_nans>= nan_threshold].index

In [19]:
nans_80 = [feature for feature in pay_nans_80 if feature in def_nans_80]

In [20]:
## check whether features were already removed
red_features_nan = [feature for feature in nans_80 if feature not in red_features] 

In [21]:
X_red = X_red.drop(columns=red_features_nan)

In [22]:
dropped_columns = red_features + red_features_nan

Building the pipeline

In [23]:
num_vars = [feature for feature in X_red.columns[2:] if feature not in cat_vars] ## exclude dates and IDs (first two columns)
str_vars = [feature for feature in X_red.columns[2:] if not pd.api.types.is_numeric_dtype(X_red[feature])] ## columns that are not numeric at all 
#red_cat_vars = [feature for feature in cat_vars if feature not in dropped_columns + str_vars] ## remaining categorical variables that have no string values
red_cat_vars = [feature for feature in cat_vars if feature not in dropped_columns + str_vars] ## remaining categorical variables 

mark rows in categorical columns with values of -1 as NaNs

In [24]:
#X_red[red_cat_vars] = X_red[red_cat_vars].applymap(lambda x: np.nan if x in [-1,-1.0, "-1.0", "-1"] else x)

In [25]:
# def nan_imp(X): ## imputes nan values for alternative values signifying nans
#     nan_list = [-1,-1.0, "-1.0", "-1"] 
#     return X.applymap(lambda x: np.nan if x in nan_list else x) ## perhaps subfunctions for arrays

In [26]:
# impute mean/most frequent value for other nans (specific to group?)
# robustscale all numerical values

num_imputer = SimpleImputer(strategy='mean')
num_scaler = RobustScaler()

#num_imputer = KNNImputer(n_neighbors=2) ## KNNIMputer is computationally demanding
## should come AFTER SCALING

num_pipe = make_pipeline(num_imputer, num_scaler)

#str_trans = OrdinalEncoder() # is only needed if one wants to do knnimputer

#nan_trans = FunctionTransformer(nan_imp)

#nan_trans = FunctionTransformer(lambda X: X.applymap(lambda x: np.nan if x in [-1,-1.0, "-1.0", "-1"] else x))

cat_imputer = SimpleImputer(strategy="most_frequent") ## replace with KNNimputer on one neighbour, after transforming to numericals
#cat_imputer = KNNImputer(n_neighbors=1) # introducing it did not improve performance, but is computationally demanding
cat_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore') ## what happens to the old columns?
cat_pipe = make_pipeline(cat_imputer, cat_encoder)
#str_pipe = make_pipeline(cat_imputer, str_trans, cat_encoder)
#str_pipe = make_pipeline(cat_imputer, cat_encoder)

Beware of the Dummy trap. OneHotEncoder automatically deletes one column.

# A self-contained alternative Nans imputer

In [27]:
X_red

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_130,D_131,D_132,D_133,R_28,D_140,D_142,D_143,D_144,D_145
0,001c886240970cf7f9c6092f0f8bda6d17100d4ac244d5...,2017-03-22,0.306287,0.004328,0.002342,0.817823,0.004932,0.174028,0.004093,0.002467,...,1.007440,0.982399,0.140760,0.297387,0.002131,1.000374,0.208669,1.004377,0.001993,0.099668
1,001c886240970cf7f9c6092f0f8bda6d17100d4ac244d5...,2017-04-21,0.331729,0.008962,0.004118,0.815960,0.000864,0.178071,0.005532,0.007005,...,1.007855,0.978045,0.158107,0.347387,0.007908,1.001228,0.211385,1.007150,0.009791,0.096142
2,001c886240970cf7f9c6092f0f8bda6d17100d4ac244d5...,2017-05-22,0.442253,0.004988,0.006820,0.813529,0.002192,0.175876,0.004110,0.000196,...,1.006285,0.978281,0.095810,0.007599,0.009932,1.006449,0.221518,1.003794,0.008036,0.094994
3,001c886240970cf7f9c6092f0f8bda6d17100d4ac244d5...,2017-06-21,0.399138,0.002508,0.004815,0.819098,0.002756,NaN,0.006862,0.000596,...,1.006142,0.984580,0.111442,0.005991,0.002849,1.001903,0.221578,1.006649,0.007658,0.097426
4,001c886240970cf7f9c6092f0f8bda6d17100d4ac244d5...,2017-07-22,0.418534,0.007389,0.006309,0.810148,0.005980,NaN,0.005039,0.007824,...,1.002154,0.980973,0.126756,0.300302,0.008404,1.001812,0.224651,1.007119,0.006605,0.095697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320695,b156cccd7100b49a211d57cbc9f16d38c3a02be530e25a...,2017-05-14,1.008203,0.006076,0.038723,1.003439,0.009835,0.166981,0.004738,0.004298,...,0.006187,0.007568,NaN,0.006808,0.000602,0.005423,NaN,0.002738,0.006319,0.005270
320696,b156cccd7100b49a211d57cbc9f16d38c3a02be530e25a...,2017-06-22,1.002040,0.008088,0.007750,1.008569,0.000236,0.176760,0.004469,0.000366,...,0.001321,0.007559,NaN,0.002433,0.002121,0.009012,NaN,0.006711,0.008114,0.009689
320697,b156cccd7100b49a211d57cbc9f16d38c3a02be530e25a...,2017-07-19,1.008051,0.064230,0.035856,1.008295,0.003580,0.154719,0.003190,0.006797,...,0.003084,0.008651,NaN,0.007968,0.000124,0.009135,NaN,0.007162,0.009125,0.001692
320698,b156cccd7100b49a211d57cbc9f16d38c3a02be530e25a...,2017-08-27,1.005604,0.008808,0.019870,1.007292,0.005523,0.165092,0.000433,0.024163,...,0.003786,0.003007,NaN,0.000611,0.000118,0.009117,NaN,0.003799,0.008319,0.004135


In [28]:
import pandas as pd
import numpy as np

def alt_nan_imp(X):
    
    cat_vars = ['B_30', 
            'B_38', 
            'D_114', 
            'D_116', 
            'D_117', 
            'D_120', 
            'D_126', 
            'D_63', 
            'D_64', 
            'D_66', 
            'D_68']
    
    alt_nan_list = [-1,-1.0, "-1.0", "-1"]
    
    cat_columns = [column for column in X.columns if column in cat_vars]
    
    X[cat_columns] = X[cat_columns].applymap(lambda x: np.nan if x in alt_nan_list else x)

alt_nan_imp(X_red)

In [29]:
preprocessor = ColumnTransformer([
    ('num_pip', num_pipe, num_vars),
    ('cat_pip', cat_pipe, red_cat_vars)],
    remainder='drop' ## all columns not in num_vars and red_cat_vars are dropped.
)

In [30]:
alt_nan_imp(X)
preprocessor.fit(X)

ColumnTransformer(transformers=[('num_pip',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('robustscaler',
                                                  RobustScaler())]),
                                 ['P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3',
                                  'D_41', 'B_3', 'D_42', 'D_43', 'D_44', 'B_4',
                                  'D_45', 'B_5', 'R_2', 'D_46', 'D_47', 'D_48',
                                  'D_49', 'B_6', 'B_7', 'B_8', 'D_50', 'D_51',
                                  'B_9', 'R_3', 'D_52', 'P_3', 'B_10', 'D_53', ...]),
                                ('cat_pip',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['B_30', 'B_38', 'D_114', 'D_116', 'D_117',
                                  'D_120', 'D_126', 'D_68'])])

In [31]:
X_pp = pd.DataFrame(preprocessor.fit_transform(X_red))

In [32]:
X_pp['customer_ID'] = X_red['customer_ID']

In [33]:
X_pp_avg = X_pp.groupby('customer_ID').mean()

In [34]:
y_ID = pd.DataFrame(y)

In [35]:
y_ID['customer_ID'] = X_red['customer_ID']

In [36]:
y_unique = y_ID.groupby('customer_ID').mean().astype(int) ## actually, this data is just in train_labels

In [37]:
X_pp_avg_train, X_pp_avg_val, y_unique_train, y_unique_val = train_test_split(X_pp_avg, y_unique, test_size=0.3) 

In [38]:
X_red[red_cat_vars].nunique().sum() ## number of unqie values in all cat. values
## should equal the number of new onehotencoded columns - 1*number of features. The latter to avoid Dummy Trap.


31

Building the simple model and putting it into the pipe

In [39]:
if 'customer_ID' in X_pp.columns:
    X_pp.drop(columns='customer_ID', inplace=True)
else:
    pass

In [40]:
X_pp.shape, y.shape

((320700, 182), (320700,))

In [41]:
mod = LogisticRegression() 
# such a model treats all rows as independent despite them being from the same person
results = cross_validate(mod, 
                         X_pp, 
                         y, 
                         cv = 5, 
                         scoring=['accuracy', 'recall', 'f1'])

In [42]:
results

{'fit_time': array([4.38956594, 3.92760968, 4.15922189, 4.58182788, 4.44010496]),
 'score_time': array([0.03167009, 0.03134012, 0.03112602, 0.03139591, 0.03392887]),
 'test_accuracy': array([0.81950421, 0.83599938, 0.81381353, 0.83537574, 0.81153726]),
 'test_recall': array([0.65504886, 0.64508143, 0.63919474, 0.64864161, 0.63873868]),
 'test_f1': array([0.63464512, 0.65310161, 0.62165758, 0.65347379, 0.61862696])}

In [43]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)

# Train_test_split needs to be on preprocessed data


In [44]:
pp_pred_pipe = make_pipeline(preprocessor, mod)

In [45]:
pp_pred_pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num_pip',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  ['P_2', 'D_39', 'B_1', 'B_2',
                                                   'R_1', 'S_3', 'D_41', 'B_3',
                                                   'D_42', 'D_43', 'D_44',
                                                   'B_4', 'D_45', 'B_5', 'R_2',
                                                   'D_46', 'D_47', 'D_48',
                                                   'D_49', 'B_6', 'B_7', 'B_8',
                                                   'D_50', 'D_51', 'B_9', 'R_3',
                                                   'D_52', 'P_3', 'B_10',
                                                   'D_53', ...]),
                                                 ('cat_pip',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['B_30', 'B_38', 'D_114',
                                                   'D_116', 'D_117', 'D_120',
                                                   'D_126', 'D_68'])])),
                ('logisticregression', LogisticRegression())])

In [46]:
pp_pred_pipe.fit(X, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num_pip',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  ['P_2', 'D_39', 'B_1', 'B_2',
                                                   'R_1', 'S_3', 'D_41', 'B_3',
                                                   'D_42', 'D_43', 'D_44',
                                                   'B_4', 'D_45', 'B_5', 'R_2',
                                                   'D_46', 'D_47', 'D_48',
                                                   'D_49', 'B_6', 'B_7', 'B_8',
                                                   'D_50', 'D_51', 'B_9', 'R_3',
                                                   'D_52', 'P_3', 'B_10',
                                                   'D_53', ...]),
                                                 ('cat_pip',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['B_30', 'B_38', 'D_114',
                                                   'D_116', 'D_117', 'D_120',
                                                   'D_126', 'D_68'])])),
                ('logisticregression', LogisticRegression())])

# Trying pickling

In [47]:
# # import pickle

# # pickle.dump(pp_pred_pipe, open('pp_pred_pipe', 'wb'))

# # loaded_model = pickle.load(open('../pickles/pp_pred_pipe', 'rb'))



# sample = X_red.sample(1)

# loaded_model.predict_proba(sample)[0][1] * 100

# Creating custom amex scoring metric

In [48]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
    
    ## TWEAK
    y_true = pd.DataFrame(y_true.reset_index(drop=True))
    y_pred = pd.DataFrame(y_pred)
    
    y_true = y_true.rename(columns={y_true.columns[0]:'target'})
    y_pred = y_pred.rename(columns={y_pred.columns[0]:'prediction'})
    ##
    
    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [49]:
from sklearn.metrics import make_scorer

In [50]:
amex_metric_scorer = make_scorer(amex_metric)

In [51]:
## dict of scoring metrics one might want to pass into cross validation
scorings = {'recall':'recall',
            'f1':'f1',
           'amex': amex_metric_scorer}

# Averaging preprocessed X and y

In [52]:
X_pp['customer_ID'] = X_red['customer_ID']

In [53]:
X_avg_pp = X_pp.groupby('customer_ID').mean()

In [54]:
y_ID = pd.DataFrame(y)

In [55]:
y_ID['customer_ID'] = X_red['customer_ID']

In [56]:
y_unique = y_ID.groupby('customer_ID').mean().astype(int) ## actually, this data is just in train_labels

In [57]:
y_unique

,default
customer_ID,
000445609ff2a39d2dd02484899affa5696210a95f6869f26390bd26eeb3b651,0
00050d84c6d26e26cd2b18c3eed83d3130c270e2361470ff272f9409103d067f,0
00055add5eaee481b27e40af3f47b8b24e82c1e550f6ab010000e7685692f281,0
0005a6ae24fd274640a237ea56c43b1ef9e32077ad168a90d464dcf2bd5976fb,0
0012e41fe6caa3ba31b55b3de2030cbb77b01203aeb4a5c6677de5b80f15cebe,0
...,...
ffeea01a48ebd3fec19fe0bbb8f88b79d8d49069d86c6bf20a54c720bae18cb5,1
fff008c2b2b68056227aa9e13c40a1773502f9d943dd46513d1364b3b68bbdef,1
fff1d5821919db2469393e4cc95a37d895bec262a723b0b4fbd25491154a9c14,0


# Trying out various models on averaged data

In [58]:
# mod = LogisticRegression() 
# ## such a model treats all rows as independent despite them being from the same person
# results = cross_validate(mod, X_avg_pp, y_unique['default'], cv = 5, scoring=scorings)

In [59]:
# results

In [60]:
# cross_val_predict(mod, X_avg_pp, y_unique['default'], cv = 5, scoring=scorings, method='predict_proba')

In [61]:
# # mod = LogisticRegression(penalty='l1',solver='liblinear') 
# ## such a model treats all rows as independent despite them being from the same person
# results = cross_validate(mod, X_avg_pp, y_unique, cv = 5, scoring=scorings)

In [62]:
# results

In [63]:
from sklearn.linear_model import Perceptron, PassiveAggressiveClassifier

In [64]:
# mod_perc = Perceptron(penalty='l1')

In [65]:
# results_perc = cross_validate(mod_perc, X_avg_pp, y_unique, cv = 5, scoring=scorings)
# results_perc['test_recall'].mean()

In [66]:
# results

In [67]:
# mod_agg = PassiveAggressiveClassifier()
# results_agg = cross_validate(mod_agg, X_avg_pp, y_unique, cv = 5, scoring=scorings)
# results_agg

In [68]:
from sklearn.neighbors import KNeighborsClassifier

In [69]:
# # knn = KNeighborsClassifier(n_neighbors=5)
# results_knn = cross_validate(knn, X_avg_pp, y_unique, cv = 5, scoring=scorings)
# results_knn['test_recall'].mean()

In [70]:
# results_knn

In [71]:
from sklearn.ensemble import GradientBoostingClassifier

In [72]:
# mod_gbc = GradientBoostingClassifier()

In [73]:
# results_gbc = cross_validate(mod_gbc, X_avg_pp, y_unique, cv = 5, scoring=scorings)
# results_gbc

In [74]:
# results_gbc

# Trying out with predict_proba

In [75]:
# pp_pred_pipe_gbc = make_pipeline(preprocessor, mod_gbc)

In [76]:
# pp_pred_pipe_gbc.fit(X_train, y_train);

In [77]:
# y_pred = pp_pred_pipe_gbc.predict_proba(X_val)[:,1]

In [78]:
# amex_metric(y_val, y_pred)

Prepare for pickle that will be used on new data

In [79]:
# mod_gbc_ws = GradientBoostingClassifier(warm_start=True) ## makes the model incrementally trainable
# pp_pred_pipe_gbc_ws = make_pipeline(preprocessor, mod_gbc_ws)
# pp_pred_pipe_gbc_ws.fit(X, y);

In [80]:
# import pickle

# pickle.dump(pp_pred_pipe_gbc_ws, open('../pickles/pp_pred_pipe_gbc_ws.pkl', 'wb'))

# # loaded_model = pickle.load(open('../pickles/pp_pred_pipe', 'rb'))



# sample = X_red.sample(1)

# loaded_model.predict_proba(sample)[0][1] * 100

In [81]:
# mod = LogisticRegression(penalty='l1',solver='liblinear') 
# pp_pred_pipe_log = make_pipeline(preprocessor, mod)
# pp_pred_pipe_log.fit(X_train, y_train);

In [82]:
# y_pred = pp_pred_pipe_log.predict_proba(X_val)[:,1]

In [83]:
# amex_metric(y_val, y_pred)

In [84]:
# mod = LogisticRegression(penalty='l1',solver='liblinear', warm_start=True) 

In [85]:
# pp_pred_pipe_log = make_pipeline(preprocessor, mod)
# pp_pred_pipe_log.(X_train, y_train);

# more efficient way to drop NANs

In [86]:
# X.dropna(axis=1, thresh=int(0.2*len(X))) 
# ## however, here you cannot specify that it should be nns of 80% in BOTH groups

# New [Light gradient boosting model](https://github.com/microsoft/LightGBM)
Todo: combine with [Optuna](https://optuna.org/)

In [98]:
import lightgbm as lgb


/Users/sjoerddewit/.pyenv/versions/3.8.12/envs/AMEX_default_prediction/lib/python3.8/site-packages/xgboost/sklearn.py:843: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Users/sjoerddewit/.pyenv/versions/3.8.12/envs/AMEX_default_prediction/lib/python3.8/site-packages/xgboost/sklearn.py:843: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Users/sjoerddewit/.pyenv/versions/3.8.12/envs/AMEX_default_prediction/lib/python3.8/site-packages/xgboost/sklearn.py:843: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Users/sjoerddewit/.pyenv/versions/3.8.12/envs

OSError: dlopen(/Users/sjoerddewit/.pyenv/versions/3.8.12/envs/AMEX_default_prediction/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: '/usr/local/opt/libomp/lib/libomp.dylib'
  Referenced from: '/Users/sjoerddewit/.pyenv/versions/3.8.12/envs/AMEX_default_prediction/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so'
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)

set_params` instead.
  warnings.warn(
/Users/sjoerddewit/.pyenv/versions/3.8.12/envs/AMEX_default_prediction/lib/python3.8/site-packages/xgboost/sklearn.py:843: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Users/sjoerddewit/.pyenv/versions/3.8.12/envs/AMEX_default_prediction/lib/python3.8/site-packages/xgboost/sklearn.py:843: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Users/sjoerddewit/.pyenv/versions/3.8.12/envs/AMEX_default_prediction/lib/python3.8/site-packages/xgboost/sklearn.py:843: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Users/s

/Users/sjoerddewit/.pyenv/versions/3.8.12/envs/AMEX_default_prediction/lib/python3.8/site-packages/xgboost/sklearn.py:843: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Users/sjoerddewit/.pyenv/versions/3.8.12/envs/AMEX_default_prediction/lib/python3.8/site-packages/xgboost/sklearn.py:843: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Users/sjoerddewit/.pyenv/versions/3.8.12/envs/AMEX_default_prediction/lib/python3.8/site-packages/xgboost/sklearn.py:843: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Users/sjoerddewit/.pyenv/versions/3.8.12/envs

/Users/sjoerddewit/.pyenv/versions/3.8.12/envs/AMEX_default_prediction/lib/python3.8/site-packages/xgboost/sklearn.py:843: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Users/sjoerddewit/.pyenv/versions/3.8.12/envs/AMEX_default_prediction/lib/python3.8/site-packages/xgboost/sklearn.py:843: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Users/sjoerddewit/.pyenv/versions/3.8.12/envs/AMEX_default_prediction/lib/python3.8/site-packages/xgboost/sklearn.py:843: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Users/sjoerddewit/.pyenv/versions/3.8.12/envs

In [ ]:
model_lgb = lgb.LGBMClassifier(boosting_type='goss', max_depth=5)

In [ ]:
model_lgb.fit(X_pp_avg_train, y_unique_train)

In [ ]:
model_lgb

In [ ]:
y_pred = model_lgb.predict_proba(X_pp_avg_val)[:,1]

In [ ]:
amex_metric(y_unique_val, y_pred)

In [ ]:
pipe_lgb = make_pipeline(preprocessor, model_lgb)

In [ ]:
alt_nan_imp(X)

In [ ]:
pipe_lgb.fit(X, y)

In [ ]:
import pickle

pickle.dump(pipe_lgb, open('../pickles/pp_pred_pipe_gbc_new1.pkl', 'wb'))

In [ ]:
y_pred = pipe_lgb.predict_proba(X_pp_avg_val)[:,1]

In [ ]:
ids = X_val['customer_ID']

In [ ]:
y_val.reset_index(drop=True)

In [ ]:
ids.reset_index(drop=True)

In [ ]:
pd.DataFrame(y_pred, columns=['prediction']).set_index(ids)

In [ ]:
amex_metric(y_val, y_pred)

In [ ]:
X_avg_pp_train, X_avg_pp_val, y_avg_train, y_avg_val = train_test_split(X_avg_pp, y_unique)

In [ ]:
model_lgb = lgb.LGBMClassifier(boosting_type='goss', max_depth=5)
model_lgb.fit(X_avg_pp_train, y_avg_train)

In [ ]:
y_pred = model_lgb.predict_proba(X_avg_pp_val)[:,1]

In [ ]:
amex_metric(y_avg_val, y_pred)

In [ ]:
pipe_lgb

# XGBoost

In [88]:
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import GridSearchCV


In [89]:
%%time

xgb_class = XGBClassifier()

# Hyperparameter Grid
grid = {'max_depth': [5, 10, 15], 
        'n_estimators': [70, 80, 90],
        'learning_rate': [0.05, 0.1, 0.15]}

# Instanciate Grid Search
search = GridSearchCV(xgb_class, 
                      grid, 
#                       scoring = 'r2',
                      cv = 5,
                      n_jobs=-1 # paralellize computation
                     ) 


search.fit(X_pp_avg_train, y_unique_train,
    # evaluate loss at each iteration
    eval_set=[(X_pp_avg_train,y_unique_train), (X_pp_avg_val, y_unique_val)],  
    # stop iterating when eval loss increases 5 times in a row
    early_stopping_rounds=4
)

[0]	validation_0-logloss:0.69263	validation_1-logloss:0.69264
[0]	validation_0-logloss:0.69263	validation_1-logloss:0.69264
[0]	validation_0-logloss:0.69263	validation_1-logloss:0.69264
[0]	validation_0-logloss:0.69263	validation_1-logloss:0.69264
[0]	validation_0-logloss:0.69263	validation_1-logloss:0.69264
[0]	validation_0-logloss:0.69263	validation_1-logloss:0.69264
[0]	validation_0-logloss:0.69263	validation_1-logloss:0.69264
[1]	validation_0-logloss:0.69212	validation_1-logloss:0.69214
[0]	validation_0-logloss:0.69263	validation_1-logloss:0.69264
[1]	validation_0-logloss:0.69212	validation_1-logloss:0.69214
[1]	validation_0-logloss:0.69212	validation_1-logloss:0.69214
[1]	validation_0-logloss:0.69212	validation_1-logloss:0.69214
[1]	validation_0-logloss:0.69212	validation_1-logloss:0.69214
[1]	validation_0-logloss:0.69212	validation_1-logloss:0.69214
[1]	validation_0-logloss:0.69212	validation_1-logloss:0.69214
[2]	validation_0-logloss:0.69161	validation_1-logloss:0.69164
[2]	vali

[18]	validation_0-logloss:0.68358	validation_1-logloss:0.68376
[18]	validation_0-logloss:0.68357	validation_1-logloss:0.68374
[18]	validation_0-logloss:0.68353	validation_1-logloss:0.68369
[19]	validation_0-logloss:0.68303	validation_1-logloss:0.68322
[19]	validation_0-logloss:0.68303	validation_1-logloss:0.68321
[19]	validation_0-logloss:0.68309	validation_1-logloss:0.68327
[18]	validation_0-logloss:0.68354	validation_1-logloss:0.68372
[18]	validation_0-logloss:0.68358	validation_1-logloss:0.68376
[19]	validation_0-logloss:0.68308	validation_1-logloss:0.68326
[20]	validation_0-logloss:0.68253	validation_1-logloss:0.68273
[19]	validation_0-logloss:0.68303	validation_1-logloss:0.68321
[19]	validation_0-logloss:0.68308	validation_1-logloss:0.68326
[19]	validation_0-logloss:0.68305	validation_1-logloss:0.68324
[20]	validation_0-logloss:0.68254	validation_1-logloss:0.68272
[20]	validation_0-logloss:0.68259	validation_1-logloss:0.68279
[19]	validation_0-logloss:0.68309	validation_1-logloss:

[52]	validation_0-logloss:0.66729	validation_1-logloss:0.66776
[53]	validation_0-logloss:0.66672	validation_1-logloss:0.66717
[54]	validation_0-logloss:0.66639	validation_1-logloss:0.66689
[54]	validation_0-logloss:0.66626	validation_1-logloss:0.66671
[53]	validation_0-logloss:0.66683	validation_1-logloss:0.66730
[55]	validation_0-logloss:0.66593	validation_1-logloss:0.66644
[55]	validation_0-logloss:0.66579	validation_1-logloss:0.66626
[54]	validation_0-logloss:0.66637	validation_1-logloss:0.66685
[56]	validation_0-logloss:0.66533	validation_1-logloss:0.66580
[56]	validation_0-logloss:0.66547	validation_1-logloss:0.66599
[55]	validation_0-logloss:0.66591	validation_1-logloss:0.66640
[57]	validation_0-logloss:0.66487	validation_1-logloss:0.66535
[57]	validation_0-logloss:0.66501	validation_1-logloss:0.66554
[56]	validation_0-logloss:0.66545	validation_1-logloss:0.66594
[58]	validation_0-logloss:0.66441	validation_1-logloss:0.66490
[57]	validation_0-logloss:0.66499	validation_1-logloss:

[19]	validation_0-logloss:0.68303	validation_1-logloss:0.68321
[19]	validation_0-logloss:0.68308	validation_1-logloss:0.68326
[19]	validation_0-logloss:0.68303	validation_1-logloss:0.68322
[19]	validation_0-logloss:0.68309	validation_1-logloss:0.68327
[21]	validation_0-logloss:0.68206	validation_1-logloss:0.68227
[20]	validation_0-logloss:0.68254	validation_1-logloss:0.68272
[20]	validation_0-logloss:0.68258	validation_1-logloss:0.68277
[20]	validation_0-logloss:0.68253	validation_1-logloss:0.68273
[22]	validation_0-logloss:0.68156	validation_1-logloss:0.68179
[21]	validation_0-logloss:0.68204	validation_1-logloss:0.68223
[20]	validation_0-logloss:0.68259	validation_1-logloss:0.68279
[21]	validation_0-logloss:0.68209	validation_1-logloss:0.68229
[23]	validation_0-logloss:0.68107	validation_1-logloss:0.68130
[21]	validation_0-logloss:0.68204	validation_1-logloss:0.68225
[21]	validation_0-logloss:0.68210	validation_1-logloss:0.68231
[22]	validation_0-logloss:0.68155	validation_1-logloss:

[27]	validation_0-logloss:0.67260	validation_1-logloss:0.67554
[27]	validation_0-logloss:0.67264	validation_1-logloss:0.67555
[29]	validation_0-logloss:0.67120	validation_1-logloss:0.67428
[29]	validation_0-logloss:0.67155	validation_1-logloss:0.67456
[36]	validation_0-logloss:0.66637	validation_1-logloss:0.67029
[24]	validation_0-logloss:0.67477	validation_1-logloss:0.67735
[27]	validation_0-logloss:0.67272	validation_1-logloss:0.67569
[24]	validation_0-logloss:0.67507	validation_1-logloss:0.67759
[28]	validation_0-logloss:0.67193	validation_1-logloss:0.67494
[28]	validation_0-logloss:0.67188	validation_1-logloss:0.67493
[30]	validation_0-logloss:0.67050	validation_1-logloss:0.67367
[30]	validation_0-logloss:0.67086	validation_1-logloss:0.67396
[25]	validation_0-logloss:0.67405	validation_1-logloss:0.67673
[37]	validation_0-logloss:0.66567	validation_1-logloss:0.66969
[28]	validation_0-logloss:0.67200	validation_1-logloss:0.67509
[29]	validation_0-logloss:0.67116	validation_1-logloss:

...

[0]	validation_0-logloss:0.27268	validation_1-logloss:0.37364
[32]	validation_0-logloss:0.25701	validation_1-logloss:0.25547
[19]	validation_0-logloss:0.15249	validation_1-logloss:0.29446
[1]	validation_0-logloss:0.19023	validation_1-logloss:0.35732
[33]	validation_0-logloss:0.25625	validation_1-logloss:0.25566
[42]	validation_0-logloss:0.08188	validation_1-logloss:0.25866
[34]	validation_0-logloss:0.25547	validation_1-logloss:0.25461
[35]	validation_0-logloss:0.25468	validation_1-logloss:0.25459
[36]	validation_0-logloss:0.25412	validation_1-logloss:0.25550
[37]	validation_0-logloss:0.25351	validation_1-logloss:0.25473
[38]	validation_0-logloss:0.25294	validation_1-logloss:0.25493
[39]	validation_0-logloss:0.25235	validation_1-logloss:0.25522
[30]	validation_0-logloss:0.10695	validation_1-logloss:0.26720
[1]	validation_0-logloss:0.19275	validation_1-logloss:0.33916
[47]	validation_0-logloss:0.07629	validation_1-logloss:0.26067
[2]	validation_0-logloss:0.15580	validation_1-logloss:0.35

[42]	validation_0-logloss:0.08072	validation_1-logloss:0.26221
[1]	validation_0-logloss:0.15300	validation_1-logloss:0.35299
[2]	validation_0-logloss:0.11668	validation_1-logloss:0.35026
[1]	validation_0-logloss:0.15144	validation_1-logloss:0.36831
[0]	validation_0-logloss:0.24290	validation_1-logloss:0.39166
[6]	validation_0-logloss:0.08543	validation_1-logloss:0.35663
[2]	validation_0-logloss:0.11791	validation_1-logloss:0.35377
[43]	validation_0-logloss:0.07966	validation_1-logloss:0.26216
[2]	validation_0-logloss:0.12093	validation_1-logloss:0.34775
[2]	validation_0-logloss:0.11569	validation_1-logloss:0.34394
[3]	validation_0-logloss:0.10040	validation_1-logloss:0.35780
[2]	validation_0-logloss:0.11724	validation_1-logloss:0.35281
[1]	validation_0-logloss:0.15228	validation_1-logloss:0.35836
[44]	validation_0-logloss:0.07868	validation_1-logloss:0.26163
[3]	validation_0-logloss:0.10141	validation_1-logloss:0.35572
[3]	validation_0-logloss:0.10674	validation_1-logloss:0.35084
[4]	v

[8]	validation_0-logloss:0.08185	validation_1-logloss:0.37713
[1]	validation_0-logloss:0.15370	validation_1-logloss:0.35763
[4]	validation_0-logloss:0.09353	validation_1-logloss:0.34955
[3]	validation_0-logloss:0.09865	validation_1-logloss:0.34793
[3]	validation_0-logloss:0.10050	validation_1-logloss:0.35943
[4]	validation_0-logloss:0.09423	validation_1-logloss:0.35444
[3]	validation_0-logloss:0.10125	validation_1-logloss:0.34663
[4]	validation_0-logloss:0.09912	validation_1-logloss:0.35945
[4]	validation_0-logloss:0.08997	validation_1-logloss:0.35272
[4]	validation_0-logloss:0.09174	validation_1-logloss:0.36368
[5]	validation_0-logloss:0.08885	validation_1-logloss:0.35100
[2]	validation_0-logloss:0.11822	validation_1-logloss:0.34605
[5]	validation_0-logloss:0.08922	validation_1-logloss:0.35793
[4]	validation_0-logloss:0.09353	validation_1-logloss:0.34955
[5]	validation_0-logloss:0.09475	validation_1-logloss:0.36370
[5]	validation_0-logloss:0.08528	validation_1-logloss:0.35568
[6]	vali

[48]	validation_0-logloss:0.06246	validation_1-logloss:0.25323
[49]	validation_0-logloss:0.06150	validation_1-logloss:0.25320
[50]	validation_0-logloss:0.06058	validation_1-logloss:0.25312
[51]	validation_0-logloss:0.05939	validation_1-logloss:0.25278
[52]	validation_0-logloss:0.05802	validation_1-logloss:0.25271
[53]	validation_0-logloss:0.05710	validation_1-logloss:0.25267
[54]	validation_0-logloss:0.05577	validation_1-logloss:0.25267
[55]	validation_0-logloss:0.05488	validation_1-logloss:0.25258
[56]	validation_0-logloss:0.05415	validation_1-logloss:0.25278
[57]	validation_0-logloss:0.05363	validation_1-logloss:0.25284
[58]	validation_0-logloss:0.05280	validation_1-logloss:0.25267
[59]	validation_0-logloss:0.05139	validation_1-logloss:0.25262
[60]	validation_0-logloss:0.05045	validation_1-logloss:0.25237
[61]	validation_0-logloss:0.04974	validation_1-logloss:0.25242
[62]	validation_0-logloss:0.04862	validation_1-logloss:0.25246
[63]	validation_0-logloss:0.04771	validation_1-logloss:

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.01, 0.1, 1],
                         'max_depth': [1, 10, 20, 50],
                         'n_estimators': [50, 80, 100]})

In [90]:
search.best_score_


0.8836577630769218

In [91]:
search.best_params_


{'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 80}

In [92]:
search.best_estimator_


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=10, max_leaves=0,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=80, n_jobs=0, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, ...)

In [94]:
xgb_best_reg = XGBClassifier(max_depth= 10,
                             n_estimators= 80, 
                             learning_rate= 0.1 )

xgb_best_reg.fit(X_pp_avg_train, y_unique_train,
    # evaluate loss at each iteration
    eval_set=[(X_pp_avg_train,y_unique_train), (X_pp_avg_val, y_unique_val)],  
    # stop iterating when eval loss increases 5 times in a row
    early_stopping_rounds=7
)



[0]	validation_0-logloss:0.61948	validation_1-logloss:0.63221
[1]	validation_0-logloss:0.55738	validation_1-logloss:0.58235
[2]	validation_0-logloss:0.50551	validation_1-logloss:0.54066
[3]	validation_0-logloss:0.46088	validation_1-logloss:0.50490
[4]	validation_0-logloss:0.42225	validation_1-logloss:0.47512
[5]	validation_0-logloss:0.38766	validation_1-logloss:0.44916
[6]	validation_0-logloss:0.35770	validation_1-logloss:0.42671
[7]	validation_0-logloss:0.33060	validation_1-logloss:0.40690
[8]	validation_0-logloss:0.30685	validation_1-logloss:0.39009
[9]	validation_0-logloss:0.28552	validation_1-logloss:0.37495
[10]	validation_0-logloss:0.26690	validation_1-logloss:0.36140
[11]	validation_0-logloss:0.24961	validation_1-logloss:0.34945
[12]	validation_0-logloss:0.23409	validation_1-logloss:0.33868
[13]	validation_0-logloss:0.21984	validation_1-logloss:0.32936
[14]	validation_0-logloss:0.20757	validation_1-logloss:0.32121
[15]	validation_0-logloss:0.19579	validation_1-logloss:0.31371
[1

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=10, max_leaves=0,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=80, n_jobs=0, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, ...)

In [96]:
y_pred = xgb_best_reg.predict(X_pp_avg_val)


In [97]:
amex_metric(y_unique_val, y_pred)

0.5537148619960569

/Users/sjoerddewit/.pyenv/versions/3.8.12/envs/AMEX_default_prediction/lib/python3.8/site-packages/xgboost/sklearn.py:843: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Users/sjoerddewit/.pyenv/versions/3.8.12/envs/AMEX_default_prediction/lib/python3.8/site-packages/xgboost/sklearn.py:843: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Users/sjoerddewit/.pyenv/versions/3.8.12/envs/AMEX_default_prediction/lib/python3.8/site-packages/xgboost/sklearn.py:843: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Users/sjoerddewit/.pyenv/versions/3.8.12/envs

/Users/sjoerddewit/.pyenv/versions/3.8.12/envs/AMEX_default_prediction/lib/python3.8/site-packages/xgboost/sklearn.py:843: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Users/sjoerddewit/.pyenv/versions/3.8.12/envs/AMEX_default_prediction/lib/python3.8/site-packages/xgboost/sklearn.py:843: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Users/sjoerddewit/.pyenv/versions/3.8.12/envs/AMEX_default_prediction/lib/python3.8/site-packages/xgboost/sklearn.py:843: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Users/sjoerddewit/.pyenv/versions/3.8.12/envs

## XGBoost & Optuna

In [ ]:
# optuna

def objective(trial):
    
    param = {
        'booster':'gbtree',
        'tree_method':'gpu_hist', 
        "objective": "binary:logistic",
        'lambda': trial.suggest_loguniform(
            'lambda', 1e-3, 10.0
        ),
#         'alpha': trial.suggest_loguniform(
#             'alpha', 1e-3, 10.0
#         ),
#         'colsample_bytree': trial.suggest_float(
#             'colsample_bytree', 0.5,1,step=0.1
#         ),
#         'subsample': trial.suggest_float(
#             'subsample', 0.5,1,step=0.1
#         ),
        'learning_rate': trial.suggest_float(
            'learning_rate', 0.001,0.05,step=0.001
        ),
        'n_estimators': trial.suggest_int(
            "n_estimators", 80,1000,10
        ),
        'max_depth': trial.suggest_int(
            'max_depth', 2,10,1
        ),
#         'random_state': 99,
#         'min_child_weight': trial.suggest_int(
#             'min_child_weight', 1,256,1
#         ),
    }
    
    xgb_reg = XGBRegressor(**param, 
#                             enable_categorical = True
                           ) 
    
    xgb_reg.fit(X_pp_avg_train, 
                y_unique_train,
    # evaluate loss at each iteration
    eval_set=[(X_pp_avg_train,y_unique_train), 
              (X_pp_avg_val, y_unique_val)],  
    # stop iterating when eval loss increases 5 times in a row
    early_stopping_rounds=5
               )

    
    xgb_reg.predict(X_pp_avg_val)
    
    amex_metric = amex_metric(y_unique_val, y_pred)
    
#     amex_metric(y_unique_val, y_pred)
                                              
    return amex_metric

In [ ]:
%%time
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials= 10)

## XGBoost new params

In [ ]:
import xgboost as xgb

In [ ]:
import pandas as pd, numpy as np # CPU libraries
import matplotlib.pyplot as plt, gc, os
from sklearn.model_selection import KFold


In [ ]:
# VERSION NAME FOR SAVED MODEL FILES
VER = 1

# TRAIN RANDOM SEED
SEED = 42

# FILL NAN VALUE
NAN_VALUE = -127 # will fit in int8

# FOLDS PER MODEL
FOLDS = 5


In [ ]:
xgb_parms = { 
    'max_depth':4, 
    'learning_rate':0.05, 
    'subsample':0.8,
    'colsample_bytree':0.6, 
    'eval_metric':'logloss',
    'objective':'binary:logistic',
    'tree_method':'gpu_hist',
    'predictor':'gpu_predictor',
    'random_state':SEED
}

In [ ]:
class IterLoadForDMatrix(xgb.core.DataIter):
    def __init__(self, df=None, features=None, target=None, batch_size=256*1024):
        self.features = features
        self.target = target
        self.df = df
        self.it = 0 # set iterator to 0
        self.batch_size = batch_size
        self.batches = int( np.ceil( len(df) / self.batch_size ) )
        super().__init__()

    def reset(self):
        '''Reset the iterator'''
        self.it = 0

    def next(self, input_data):
        '''Yield next batch of data.'''
        if self.it == self.batches:
            return 0 # Return 0 when there's no more batch.
        
        a = self.it * self.batch_size
        b = min( (self.it + 1) * self.batch_size, len(self.df) )
        dt = cudf.DataFrame(self.df.iloc[a:b])
        input_data(data=dt[self.features], label=dt[self.target]) #, weight=dt['weight'])
        self.it += 1
        return 1

In [ ]:
train = X_pp_avg_train
TRAIN_SUBSAMPLE = 1.0


In [ ]:
importances = []
oof = []

skf = KFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
for fold,(train_idx, valid_idx) in enumerate(skf.split(
            train, y_unique_train )):
    
    # TRAIN WITH SUBSAMPLE OF TRAIN FOLD DATA
    if TRAIN_SUBSAMPLE<1.0:
        np.random.seed(SEED)
        train_idx = np.random.choice(train_idx, 
                       int(len(train_idx)*TRAIN_SUBSAMPLE), replace=False)
        np.random.seed(None)
    
    print('#'*25)
    print('### Fold',fold+1)
    print('### Train size',len(train_idx),'Valid size',len(valid_idx))
    print(f'### Training with {int(TRAIN_SUBSAMPLE*100)}% fold data...')
    print('#'*25)
    
    # TRAIN, VALID, TEST FOR FOLD K
    Xy_train = IterLoadForDMatrix(train.loc[train_idx], FEATURES, 'target')
    X_valid = train.loc[valid_idx, FEATURES]
    y_valid = train.loc[valid_idx, 'target']
    
    dtrain = xgb.DeviceQuantileDMatrix(Xy_train, max_bin=256)
    dvalid = xgb.DMatrix(data=X_valid, label=y_valid)
    
    # TRAIN MODEL FOLD K
    model = xgb.train(xgb_parms, 
                dtrain=dtrain,
                evals=[(dtrain,'train'),(dvalid,'valid')],
                num_boost_round=9999,
                early_stopping_rounds=100,
                verbose_eval=100) 
    model.save_model(f'XGB_v{VER}_fold{fold}.xgb')
    
    # GET FEATURE IMPORTANCE FOR FOLD K
    dd = model.get_score(importance_type='weight')
    df = pd.DataFrame({'feature':dd.keys(),f'importance_{fold}':dd.values()})
    importances.append(df)
            
    # INFER OOF FOLD K
    oof_preds = model.predict(dvalid)
    acc = amex_metric_mod(y_valid.values, oof_preds)
    print('Kaggle Metric =',acc,'\n')
    
    # SAVE OOF
    df = train.loc[valid_idx, ['customer_ID','target'] ].copy()
    df['oof_pred'] = oof_preds
    oof.append( df )
    
    del dtrain, Xy_train, dd, df
    del X_valid, y_valid, dvalid, model
    _ = gc.collect()
    
print('#'*25)
oof = pd.concat(oof,axis=0,ignore_index=True).set_index('customer_ID')
acc = amex_metric_mod(oof.target.values, oof.oof_pred.values)
print('OVERALL CV Kaggle Metric =',acc)

